# Data reading and cleaning 

## Import libraries

In [2]:
import pandas as pd
import numpy as np 
import re

## Functions
getProducts(path) <br>
getEncodedProducts(path)

### Read producto_tabla.csv

#### Create categories by product

In [5]:
def get_category(name):
    name = re.sub(r'\d+[gpKmLlin].*$', '', name)
    return name

#### Get pieces of product

In [6]:
def df_add_piece_amount(df):
    pieces = df['NombreProducto'].str.extract(' (\d+)(p|pct)', expand=True)[0]
    pieces.fillna(1, inplace=True)
    df['PieceAmount'] = pieces.astype(int)
    return df

#### Get weight of product

In [7]:
def df_add_weight_grams(df):
    weights = df['NombreProducto'].str.extract(r' (\d+)(ml|g|Kg|kg)', expand=True)
    #Tostada Ondulada Tubo is the only product with no weight in the name, searched for it and found it was 360g
    weights[0].fillna(360,inplace=True)
    weights[1].fillna('g',inplace=True)
    weights[0] = weights[0].astype(int)
    weights[1] = weights[1].astype(str)
    weights[1] = weights[1].str.lower()
    weights[0] = np.where(weights[1] == 'kg', weights[0]*1000, weights[0])

    df['WeightGrams'] = weights[0]
    return df

In [8]:
def setCategoryID(df):
    cat_names = list(df['CategoriaProducto'].unique())
    categories = {}

    for product in cat_names:
        categories[product] = cat_names.index(product)
        
    df['CategoriaID'] = df['CategoriaProducto'].map(categories)
    return df


#### **getProducts(path): Return products dataframe with productId, product category (object), weight and pieces**

In [9]:
def getProducts(path = 'data/producto_tabla.csv'): 
    productsDF = pd.read_csv(path)
    
    productsDF['CategoriaProducto'] = productsDF['NombreProducto'].apply(get_category)
    productsDF = setCategoryID(productsDF)
    productsDF = df_add_weight_grams(productsDF)
    productsDF = df_add_piece_amount(productsDF)
    productsDF = productsDF.drop(['NombreProducto'], axis=1)

    return productsDF

productsDF = getProducts()
productsDF.head()


,Producto_ID,CategoriaProducto,CategoriaID,WeightGrams,PieceAmount
0,0,NO IDENTIFICADO 0,0,360,1
1,9,Capuccino Moka,1,750,1
2,41,Bimbollos Ext sAjonjoli,2,480,6
3,53,Burritos Sincro,3,170,1
4,72,Div Tira Mini Doradita,4,45,4


#### **getEncodedProducts(path): Return ENCODED products dataframe with productId, product category (encoded), weight and pieces**

In [24]:
def getEncodedProducts(path = 'data/producto_tabla.csv'): 
    productsDF = pd.read_csv(path)
    
    productsDF['CategoriaProducto'] = productsDF['NombreProducto'].apply(get_category)
    productsDF = df_add_weight_grams(productsDF)
    productsDF = df_add_piece_amount(productsDF)
    productsDF = productsDF.drop(['NombreProducto'], axis=1)
    productsDF = pd.get_dummies(productsDF, columns=['CategoriaProducto'])

    return productsDF

productsDF = getEncodedProducts()
productsDF.head()


,Producto_ID,WeightGrams,PieceAmount,CategoriaProducto_,CategoriaProducto_12 Tarima MiniMilk Kitty Nav,CategoriaProducto_12Granos Multigra TwinPack,CategoriaProducto_12Tarima Mini Milk Kitty SnVal,CategoriaProducto_7 Granos,CategoriaProducto_ActiFresh Menta,CategoriaProducto_ActiFresh Yerbabuena,...,CategoriaProducto_Tuinky Vainilla,CategoriaProducto_Twin Pack Thins Multig,CategoriaProducto_Twinkies Vainilla,CategoriaProducto_Two Pack Classicas AvellanaFsa,CategoriaProducto_Unico Fresco Naranja,CategoriaProducto_Valentones,CategoriaProducto_Vasos 226,CategoriaProducto_Vidrio Congelado,CategoriaProducto_Wonder,CategoriaProducto_Wonderbutter
0,0,360,1,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,9,750,1,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,41,480,6,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,53,170,1,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,72,45,4,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


### Read town_state.csv

#### readTown(path)
Return dataframe with the town_state.csv data 

In [4]:
def readTown(path = 'data/town_state.csv'):
    townDF = pd.read_csv(path)
    return townDF

#### getTownAgencies(path, town)
Return list with the Agencia_ID of the specified town

In [3]:
def getTownAgencies(path = 'data/town_state.csv', town = 'AGUASCALIENTES'):
    townDF = pd.read_csv(path)
    agencies = townDF.loc[townDF['State'] == town]['Agencia_ID'].unique()
    return agencies
getTownAgencies(town = 'AGUASCALIENTES')

array([ 1347,  1351,  1355,  1374,  1974, 22519, 22529], dtype=int64)

### Differenced Demanda_uni_equil

#### getDifferencedDemand(df, lag = 2)
Get a dataframe with the differenced Demanda_uni_equil according to the lag periods

In [10]:
def getDifferencedDemand(df, lag = 2): 
    demand_pivot = df.pivot_table(index=['Semana', 'Cliente_ID', 'Producto_ID', 'Agencia_ID'], values='Demanda_uni_equil', aggfunc='sum')
    for i in range(lag):
        demand_pivot[f'Unit_Diff{i + 1}_Demanda_uni_equil'] = demand_pivot.groupby(['Producto_ID', 'Agencia_ID','Cliente_ID'])['Demanda_uni_equil'].diff(periods=i+1)
    
    diff_df = df.merge(demand_pivot.drop(['Demanda_uni_equil'], axis=1), on=['Semana', 'Cliente_ID', 'Producto_ID', 'Agencia_ID'], how='left')
    
    diff_df.fillna(0, inplace=True)
    
    return diff_df


In [62]:
a.query('Producto_ID == 2233 & Agencia_ID == 1347 & Cliente_ID == 9200 & (Semana == 3 | Semana == 4 | Semana == 5) ').pivot_table(index=['Semana', 'Cliente_ID', 'Producto_ID', 'Agencia_ID'], values=['Demanda_uni_equil', 'Unit_Diff1_Demanda_uni_equil', 'Unit_Diff2_Demanda_uni_equil'], aggfunc='sum')

,,,,Demanda_uni_equil,Unit_Diff1_Demanda_uni_equil,Unit_Diff2_Demanda_uni_equil
Semana,Cliente_ID,Producto_ID,Agencia_ID,,,
3,9200,2233,1347,1,0.0,0.0
4,9200,2233,1347,2,1.0,0.0
5,9200,2233,1347,4,2.0,3.0


### Read train.csv of specified town

#### Get train data with products as label encoding

In [ ]:
def getTrainData(path = 'data/train.csv', town = 'AGUASCALIENTES'):
    trainDF = pd.DataFrame()
    
    for chunk in pd.read_csv(path, chunksize=100000):
        trainDF = pd.concat([trainDF, chunk.loc[chunk['Agencia_ID'].isin(getTownAgencies(town=town))]])
    
    trainDF = getDifferencedDemand(trainDF, lag=4)
    
    productsDF = getProducts()
    
    trainDF = trainDF.merge(productsDF, on='Producto_ID', how='left')                        
    return trainDF


#### Get train data with products as dummies encoding

In [70]:
def getEncodedTrainData(path = 'data/train.csv', town = 'AGUASCALIENTES'):
    trainDF = pd.DataFrame()
    
    for chunk in pd.read_csv(path, chunksize=100000):
        trainDF = pd.concat([trainDF, chunk.loc[chunk['Agencia_ID'].isin(getTownAgencies(town=town))]])
    
    trainDF = getDifferencedDemand(trainDF, lag=4)
    
    productsDF = getEncodedProducts()
    
    trainDF = trainDF.merge(productsDF, on='Producto_ID', how='left')                        
    return trainDF

## Save data as csv file

#### Label encoded data

In [72]:
trainDF = getTrainData()
trainDF.to_csv('data/train_aguascalientes.csv', index=False)


#### Dummy encoded data 

In [73]:
trainEncodedDF = getEncodedTrainData()
trainEncodedDF.to_csv('data/train_encoded_aguascalientes.csv', index=False)

# Test data

In [ ]:
def getTestData(path_train = 'data/train.csv', path_test = 'data/test.csv', town = 'AGUASCALIENTES'):
    test = pd.DataFrame()
    
    for chunk in pd.read_csv(path_train, chunksize=100000):
        test = pd.concat([test.drop(['Demanda_uni_equil', 'CategoriaProducto', 
                                           'Venta_uni_hoy', 'Venta_hoy', 'Dev_uni_proxima', 'Dev_proxima'],
                                          axis=1), chunk.loc[chunk['Agencia_ID'].isin(getTownAgencies(town=town))]])
        
    for chunk in pd.read_csv(path_test, chunksize=100000):
        trainDF = pd.concat([trainDF, chunk.loc[chunk['Agencia_ID'].isin(getTownAgencies(town=town))]])
    
    test = getDifferencedDemand(trainDF, lag=4)
    
    productsDF = getProducts()
    
    trainDF = trainDF.merge(productsDF, on='Producto_ID', how='left')                        
    return trainDF